# ARIMA Model

In [ ]:
import pandas as pd 
from sklearn.model_selection  import train_test_split
import numpy as np 
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf, plot_predict
from pandas import DataFrame
from sklearn.metrics import mean_squared_error
from math import sqrt
from pmdarima import auto_arima
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error ,r2_score 
from statsmodels.tools.eval_measures import rmse

# Load and PreProcess 

In [11]:
data = pd.read_csv('/home/rohullah/Desktop/files/HH/MCs/data/utd19_u.csv', nrows = 10000,usecols = ['day','flow'])

In [12]:
data.head()

,day,flow
0,2017-05-06,12
1,2017-05-06,12
2,2017-05-06,12
3,2017-05-06,16
4,2017-05-06,16


In [13]:
#data['day'] = pd.to_datetime(data['day'], format="%Y-%m-%d")

In [14]:
#ratio = 0.75
  
#total_rows = data.shape[0]
#train_size = int(total_rows*ratio)
#train_set = data[0:train_size]
#test_set = data[train_size:]

In [15]:
#print(len(train_set))
#print(len(test_set))

In [16]:
#plt.plot(train_set['flow'].index,train_set['flow'].values )
#plt.xlim(0,7500)
#plt.plot(test_set['flow'].index, test_set['flow'].values)
#plt.xlim(7500,10000)
#plt.title('Data split')
#plt.xlabel('Number of data')
#plt.ylabel('Flow per Hour')


# Train ARIMA 

In [17]:
#xtrain = train_set['flow'].values
#xtest = test_set['flow'].values

In [18]:
#data_set = data['flow']
#data_set.head()

In [49]:
def evaluate_arima_model(X, arima_order):
    # prepare training dataset
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    #print(history)
    # make predictions
    predictions_eval = []
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions_eval.append(yhat)
        history.append(test[t])
    rmse = sqrt(mean_squared_error(test, predictions_eval))
    return rmse




In [51]:
def evaluate_models(dataset, p_values, d_values, q_values):
    #dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        #print(p)
        for d in d_values:
            #print(d)
            for q in q_values:
                #print(q)
                order = (p,d,q)
                #print(order)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print('ARIMA%s RMSE=%.3f' % (order,rmse))
                except:
                    print('fuck william')
                    continue
    print('Best ARIMA%s RMSE=%.3f' % (best_cfg, best_score))



In [ ]:
# evaluate parameters
p_values = [3,4,5,6,7]
d_values = range(0, 3)
q_values = range(0, 3)
model = evaluate_models(data['flow'].values, p_values, d_values, q_values)

In [ ]:
plt.plot(predictions, color='red')

In [ ]:

# plot forecasts against actual outcomes
plt.plot(xtest,color = 'blue',label = "Actual")
plt.plot(predictions, color='red', label = "Prediction")
plt.xlim(1260, 1300)
plt.legend()
plt.show()

In [ ]:
r2 = r2_score(xtest, predictions)
rmse = sqrt(mean_squared_error(xtest, predictions))
print('RMSE', rmse, 'R_2', r2)

In [ ]:
residuals = DataFrame(model_fit.resid) 
residuals.plot()
plt.show()

In [ ]:
#we continue to check residuals
residuals.plot(kind='kde') 
plt.show()